In [1]:
import sys
from pathlib import Path
from copy import deepcopy
# from os.path import exists

# Add the path to the sys.path list
sys.path.append(Path(Path.cwd(), "..", "galfitlib").as_posix())

In [2]:
#from galfitlib.classes.components import *
from galfitlib.classes.containers import *
from galfitlib.functions.helper_functions import *
from galfitlib.pipeline.s2g_feedme_gen import *

In [3]:
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from scipy import fft
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#Imports
import ipywidgets as widgets
from IPython.display import display
from IPython.display import Audio

In [5]:
import sounddevice as sd

In [6]:
# # From the twelve-tone tuning system
notes = [
    ['C0', 16.35],
    ['C#0/Db0', 17.32],
    ['D0', 18.35],
    ['D#0/Eb0', 19.45],
    ['E0', 20.60],
    ['F0', 21.83],
    ['F#0/Gb0', 23.12],
    ['G0', 24.50],
    ['G#0/Ab0', 25.96],
    ['A0', 27.50],
    ['A#0/Bb0', 29.14],
    ['B0', 30.87],
    ['C1', 32.70],
    ['C#1/Db1', 34.65],
    ['D1', 36.71],
    ['D#1/Eb1', 38.89],
    ['E1', 41.20],
    ['F1', 43.65],
    ['F#1/Gb1', 46.25],
    ['G1', 49.00],
    ['G#1/Ab1', 51.91],
    ['A1', 55.00],
    ['A#1/Bb1', 58.27],
    ['B1', 61.74],
    ['C2', 65.41],
    ['C#2/Db2', 69.30],
    ['D2', 73.42],
    ['D#2/Eb2', 77.78],
    ['E2', 82.41],
    ['F2', 87.31],
    ['F#2/Gb2', 92.50],
    ['G2', 98.00],
    ['G#2/Ab2', 103.83],
    ['A2', 110.00],
    ['A#2/Bb2', 116.54],
    ['B2', 123.47],
    ['C3', 130.81],
    ['C#3/Db3', 138.59],
    ['D3', 146.83],
    ['D#3/Eb3', 155.56],
    ['E3', 164.81],
    ['F3', 174.61],
    ['F#3/Gb3', 185.00],
    ['G3', 196.00],
    ['G#3/Ab3', 207.65],
    ['A3', 220.00],
    ['A#3/Bb3', 233.08],
    ['B3', 246.94],
    ['C4', 261.63],
    ['C#4/Db4', 277.18],
    ['D4', 293.66],
    ['D#4/Eb4', 311.13],
    ['E4', 329.63],
    ['F4', 349.23],
    ['F#4/Gb4', 369.99],
    ['G4', 392.00],
    ['G#4/Ab4', 415.30],
    ['A4', 440.00],
    ['A#4/Bb4', 466.16],
    ['B4', 493.88],
    ['C5', 523.25],
    ['C#5/Db5', 554.37],
    ['D5', 587.33],
    ['D#5/Eb5', 622.25],
    ['E5', 659.25],
    ['F5', 698.46],
    ['F#5/Gb5', 739.99],
    ['G5', 783.99],
    ['G#5/Ab5', 830.61],
    ['A5', 880.00],
    ['A#5/Bb5', 932.33],
    ['B5', 987.77],
    ['C6', 1046.50],
    ['C#6/Db6', 1108.73],
    ['D6', 1174.66],
    ['D#6/Eb6', 1244.51	],
    ['E6', 1318.51],
    ['F6', 1396.91],
    ['F#6/Gb6', 1479.98],
    ['G6', 1567.98],
    ['G#6/Ab6', 1661.22],
    ['A6', 1760.00	],
    ['A#6/Bb6', 1864.66],
    ['B6', 1975.53	],
    ['C7', 2093.00],
    ['C#7/Db7', 2217.46],
    ['D7', 2349.32],
    ['D#7/Eb7', 2489.02],
    ['E7', 2637.02],
    ['F7', 2793.83],
    ['F#7/Gb7 ', 2959.96],
    ['G7', 3135.96],
    ['G#7/Ab7', 3322.44],
    ['A7', 3520.00],
    ['A#7/Bb7', 3729.31],
    ['B7', 3951.07],
    ['C8', 4186.01],
    ['C#8/Db8', 4434.92],
    ['D8', 4698.63],
    ['D#8/Eb8', 4978.03],
    ['E8', 5274.04],
    ['F8', 5587.65],
    ['F#8/Gb8', 5919.91],
    ['G8', 6271.93],
    ['G#8/Ab8', 6644.88	],
    ['A8', 7040.00],
    ['A#8/Bb8', 7458.62],
    ['B8', 7902.13],
]

In [7]:
note_dict_ranges = {}
previous_tone_number = "0"
previous_name        = None
for name, freq in notes:
    name = name.replace("#", "").replace("b", "").strip()
    
    current_tone_number = name[-1]
    
    # There are gaps when jumping to different numbers
    if current_tone_number != previous_tone_number:
        note_dict_ranges[previous_name].append(freq)
        
    note_letter_1 = name
    note_letter_2 = None
    
    if "/" in name:
        note_letter_1, note_letter_2 = name.split("/")
        #note_letter_1 = note_letter_1[0]
        #note_letter_2 = note_letter_2[0]
        
        note_dict_ranges[note_letter_1].append(freq)
        note_dict_ranges[note_letter_2] = [freq]
        
    else:
        if note_letter_1 not in note_dict_ranges:
            note_dict_ranges[note_letter_1] = [freq]
        else:
            note_dict_ranges[note_letter_1].append(freq)
            
    previous_tone_number = current_tone_number
    previous_name        = name

In [8]:
note_dict_ranges

{'C0': [16.35, 17.32],
 'D0': [17.32, 18.35, 19.45],
 'E0': [19.45, 20.6],
 'F0': [21.83, 23.12],
 'G0': [23.12, 24.5, 25.96],
 'A0': [25.96, 27.5, 29.14],
 'B0': [29.14, 30.87, 32.7],
 'C1': [32.7, 34.65],
 'D1': [34.65, 36.71, 38.89],
 'E1': [38.89, 41.2],
 'F1': [43.65, 46.25],
 'G1': [46.25, 49.0, 51.91],
 'A1': [51.91, 55.0, 58.27],
 'B1': [58.27, 61.74, 65.41],
 'C2': [65.41, 69.3],
 'D2': [69.3, 73.42, 77.78],
 'E2': [77.78, 82.41],
 'F2': [87.31, 92.5],
 'G2': [92.5, 98.0, 103.83],
 'A2': [103.83, 110.0, 116.54],
 'B2': [116.54, 123.47, 130.81],
 'C3': [130.81, 138.59],
 'D3': [138.59, 146.83, 155.56],
 'E3': [155.56, 164.81],
 'F3': [174.61, 185.0],
 'G3': [185.0, 196.0, 207.65],
 'A3': [207.65, 220.0, 233.08],
 'B3': [233.08, 246.94, 261.63],
 'C4': [261.63, 277.18],
 'D4': [277.18, 293.66, 311.13],
 'E4': [311.13, 329.63],
 'F4': [349.23, 369.99],
 'G4': [369.99, 392.0, 415.3],
 'A4': [415.3, 440.0, 466.16],
 'B4': [466.16, 493.88, 523.25],
 'C5': [523.25, 554.37],
 'D5': [5

In [9]:
#note_dict_rangesa
ranges_for_search = [                                                    
    {
        "start" : freqs[0],                      
        "end"   : freqs[-1],                           
        "value" : note
    }                        
    for note, freqs in note_dict_ranges.items()
]
ranges_for_search

[{'start': 16.35, 'end': 17.32, 'value': 'C0'},
 {'start': 17.32, 'end': 19.45, 'value': 'D0'},
 {'start': 19.45, 'end': 20.6, 'value': 'E0'},
 {'start': 21.83, 'end': 23.12, 'value': 'F0'},
 {'start': 23.12, 'end': 25.96, 'value': 'G0'},
 {'start': 25.96, 'end': 29.14, 'value': 'A0'},
 {'start': 29.14, 'end': 32.7, 'value': 'B0'},
 {'start': 32.7, 'end': 34.65, 'value': 'C1'},
 {'start': 34.65, 'end': 38.89, 'value': 'D1'},
 {'start': 38.89, 'end': 41.2, 'value': 'E1'},
 {'start': 43.65, 'end': 46.25, 'value': 'F1'},
 {'start': 46.25, 'end': 51.91, 'value': 'G1'},
 {'start': 51.91, 'end': 58.27, 'value': 'A1'},
 {'start': 58.27, 'end': 65.41, 'value': 'B1'},
 {'start': 65.41, 'end': 69.3, 'value': 'C2'},
 {'start': 69.3, 'end': 77.78, 'value': 'D2'},
 {'start': 77.78, 'end': 82.41, 'value': 'E2'},
 {'start': 87.31, 'end': 92.5, 'value': 'F2'},
 {'start': 92.5, 'end': 103.83, 'value': 'G2'},
 {'start': 103.83, 'end': 116.54, 'value': 'A2'},
 {'start': 116.54, 'end': 130.81, 'value': 'B

In [27]:
def binary_search_ranges(ranges, values : list) -> list:
    """
    Wrapper for binary search on a list of values in a list of ranges
    """
    def binary_search(ranges, value):
        """
        Binary search for a value in a list of ranges
        Thanks to https://stackoverflow.com/a/65518827 for the code
        """
    
        if len(ranges) == 1:
            if ranges[0]["start"] <= value <= ranges[0]["end"]:
                pass
            else:
                print(f"Value ({value:.2f}) not found in *any* range.")
                print(f"Choosing note: {ranges[0]['value']}")
            return ranges[0]["value"]
    
        mid = len(ranges) // 2
        if ranges[mid]["end"] < value:
            return binary_search(ranges[mid:], value)
        elif ranges[mid]["start"] > value:
            return binary_search(ranges[:mid], value)
        else:
            return ranges[mid]["value"]
        
    return [binary_search(ranges_for_search, value) for value in values]
    #yield binary_search(ranges_for_search, value)

In [11]:
def music_to_galaxy_properties(
        bulge_object   : Sersic,
        disk_object    : Sersic,
        arms_object    : Power,
        # fourier_object : Fourier,
        dict_music     : dict
):
    """
    BULGE
    G4 – ‘Magnitude’ [10,16] -> Linearly [0,1] | Set: 15 | Direction: Longer -> Lower Value
    C5 – ‘Effective Radius’ [0, 75] | Set: 3 | Direction: Longer -> Higher Value
    E4 – ‘Sersic Index’ [0.1, 10] (log scaling) | Set: 1.0 | Direction: Longer -> Higher Value
    G5 – ‘Axis Ratio’ [0.5, 1] | Set: 1.0 | Direction: Longer -> Lower Value
    D4 – ‘Position Angle’ [0.5, 1] | Set: 1.0 | Direction: Longer -> Lower Value
    
    DISK
    G4  – ‘Magnitude’ [10, 16] -> Linearly [0, 1] | Set: 15 | Direction: Longer -> Lower Value
    RMS – ‘Eff_Rad’ [0, 75] | Set: 20 | Direction: Longer -> Higher Value
    D5  – ‘Sersic_Index’ [0.1, 10] (log scaling) | Set: 1.0 | Direction: Longer -> Higher Value
    G4  – ‘Pos_Angle’ [0, 180] -> [0,1] Sin(theta/2) | Set: 0 | Direction: Longer -> Higher Value
    B4  – ‘Axis_Ratio’ [0.5, 1] | Set: 1.0 | Direction: Longer -> Lower Value
    
    Arms
    Let position angle be handled by disk
    volume - spiral outer radius
    B4 - # of fourier modes
    C5 - cumulative rotation
    E5 - spiral power law
    D4 - spiral inner radius
    A4 - strength of fourier modes
    E4 - inclination
    """
        
    def normalize(val, val_min=1.0, val_max=5.0):
        """
        Normalizes input to [0.5,1]
        """
        norm = np.abs((val - val_min)/(val_max - val_min))
        return norm

    # The sersic_object is mutable, so we can directly update its properties
    # BULGE
    bulge_object.magnitude.value        = (16 - 10) * (1.0 - normalize(dict_music['G4'])) + 3.
    bulge_object.effective_radius.value = (75 - 0.0)*normalize(dict_music['C5']) + 10
    bulge_object.sersic_index.value     = (10.0 - 0.1)*normalize(dict_music['E4']) + 0.5
    bulge_object.axis_ratio.value       = (1 - 0.5)*normalize(dict_music['G5']) + 0.5
    bulge_object.position_angle.value   = 2 * np.arcsin(normalize(dict_music['D4'])) * 180.0/np.pi
    
    # DISK
    disk_object.magnitude.value        = (16 - 10) * (1.0 - normalize(dict_music['G4'])) + 3.
    disk_object.effective_radius.value = (75 - 0.0)*normalize(dict_music['RMS']) + 10
    disk_object.sersic_index.value     = (10.0 - 0.1)*normalize(dict_music['D5']) + 0.5
    disk_object.axis_ratio.value       = (1 - 0.5)*normalize(dict_music['G4']) + 0.5
    disk_object.position_angle.value   = 2 * np.arcsin(normalize(dict_music['B4'])) * 180.0/np.pi
        
    # ARMS
    arms_object.outer_radius.value = (75 - 0)*normalize(dict_music['RMS'])# + 10
    arms_object.cumulative_rotation.value = (360 - 0)*normalize(dict_music['C5'])# + 10
    arms_object.power_law.value = (3 - -1)*normalize(dict_music['E5'])# + 10
    arms_object.inner_radius.value = (20 - 0)*normalize(dict_music['D4'])# + 10 
    arms_object.inclination.value = (45 - 0)*normalize(dict_music['E4'])# + 10
    
    # FOURIER
    num_fourier_modes = (5 - 1)*normalize(dict_music['B4'])
    fourier_object = Fourier(num_fourier_modes)
    fourier_object.amplitude.value = (0.05 - 0.001)*normalize(dict_music['A4'])# + 10
    
    return fourier_object

In [12]:
g_prefix = "galaxy"
g_number = 0

cwd = Path.cwd()

IN_DIR  = pj(cwd, "music-in")
TMP_DIR = pj(cwd, "music-tmp")
OUT_DIR = pj(cwd, "music-out")

previous = False

In [40]:
def rms(data, width):
    """
    audioop.rms() using numpy; since audioop will be deprecated
    Thanks https://stackoverflow.com/q/9763471 !
    """
    
    if len(data) == 0: 
        return None
    
    from_type = (np.int8, np.int16, np.int32)[width//2]
    d         = np.frombuffer(data, from_type).astype(float)
    return int(np.sqrt(np.mean(d**2)))

In [13]:
# Function to update the array image when slider value changes
def update_image_array(change):
    # Update the image data based on the slider value
    new_image_data_array = image_data_array * change.new

    # Update the array image with new data
    image_array.set_data(new_image_data_array)

    # Redraw the figure
    fig.canvas.draw()

In [38]:
import librosa
x, sr = librosa.load('brown_alma_mater.mp3')

In [15]:
# import pygame
# 
# def play_audio(file_path):
#     # Initialize Pygame mixer
#     pygame.mixer.init()
# 
#     # Load the audio file
#     pygame.mixer.music.load(file_path)
# 
#     # Play the audio file
#     pygame.mixer.music.play()
# 
#     # Keep the script running while the audio is playing
#     while pygame.mixer.music.get_busy():
#         pygame.time.Clock().tick(10)
# 
# 
# audio_file = "brown_alma_mater.mp3"
# play_audio(audio_file)

In [16]:
%matplotlib inline

In [28]:
count = 0
g_number = 0
while g_number == 0:
    plt.clf()
    gname = f"{g_prefix}_{g_number}"
    filename = f"{pj(IN_DIR, gname)}.fits"
    #template = pj(_MODULE_DIR, "template.fits")
    #sp(f"cp {template} {filename}")
    #gen_template(filename)
    
    # Ideas: 
    # Coordinate notes with features of galaxies
    # Normalize range of frequencies and modify galaxy property according to position in range
    # dict_music = {
    #     'D5': 3.0,
    #     'G4': 0.5,
    #     'B4': 1.0,
    #     'G': 0.5,
    #     'D': 0.0,
    #     'M': 1,
    #     'm': 0
    # }
    # TODO: Dynamically determine frequent notes (pre-loop the song by chunks)
    # and map to galaxy properties
    # aka create an agnostic mapping simply based on frequency
    frequent_notes = ['D5', 'G4', 'B4', 'C5', 'E5', 'D4', 'E4', 'G5', 'A4', 'C4']

    if not exists(pj(OUT_DIR, gname)):
        pj(OUT_DIR, gname).mkdir()

    feedme = pj(OUT_DIR, gname, f"{gname}.in")
    input_filename = pj(IN_DIR, f"{gname}.fits")
    output_filename = pj(OUT_DIR, gname, f"{gname}_out.fits")
    
    x     = 10000
    count = 0
    filename = "brown_alma_mater.mp3"
    list_o_notes = []
    
    while x != 0 and count < 1:
        # x = int(input("Waiting..."))
        #sr = 48000
        #duration = 6
        signal, sr = librosa.load(filename)
        t = np.arange(librosa.get_duration(path = filename), step = 1/sr)
        #Nsamples = int(duration*sr)
        # Cmaj = sd.rec(Nsamples, samplerate=sr, channels=1, blocking=True)[:,0]#dur5
        
         #np.arange(Nsamples)/sr
        #extracting chord and duration
        #ignore first 1 seconds
        # blur(abs(signal)) to find peaks
        # FT first peak to get chord
        # output as string
        # time between this and last peak, output this too

        ignore = 0.5 # seconds
        signal = signal[t > 0.5]
        t      = t[t > 0.5]
        
        s = 0.01
        t_step_size = 5 * sr
        t_initial   = int(ignore * sr) + t_step_size
        
        for t_step in range(t_initial, len(t), t_step_size):
            
            #t_step = t[i_step] - t[i_step - i_step_size]
            cutsignal = signal[t_step - t_step_size : t_step]
            
            blurProfile = (1/np.sqrt(2*np.pi*s**2))*np.exp(-0.5*(np.linspace(-3*s, 3*s, int(6*s*sr))/s)**2)
    
            signalBlurred = np.convolve(cutsignal**2, blurProfile, mode='same')
    
            chordrange = signalBlurred>100
            if not np.any(chordrange):
                continue
    
            FT    = fft.fft(cutsignal[chordrange])
            freqs = fft.fftfreq(len(cutsignal[chordrange]), d=1/sr)
    
            FT    = FT[freqs >= 0]
            freqs = freqs[freqs >= 0]
            fmask = (freqs>10)&(freqs<2000)
    
            # noteBins = 220*(2**((np.arange(12*3)-0.5)/12)) # starting from A below middle
            
            # Very basic peak detection method
            peaks      = freqs[np.argsort(FT)[-5:]]
            peak_notes = binary_search_ranges(ranges_for_search, peaks)
            list_o_notes.extend(peak_notes)
            
            volume = rms(cutsignal, 2)
            
            # plt.clf()
            # fig,ax = plt.subplots()
            # ax.plot(freqs[fmask], np.abs(FT[fmask])**2)
            # plt.show()
            
            #break
        
        # bulge = Sersic(
        #         component_number = 1,
        #         position = (74, 74),
        # )
        # 
        # disk  = Sersic(
        #         component_number = 2,
        #         position = (74, 74),
        # )
        # 
        # arms  = Power(
        #         component_number = 2,
        #         position = (74, 74),
        # )
        # 
        # header = GalfitHeader(
        #         galaxy_name = gname,
        #         input_image = input_filename,
        #         output_image = output_filename,
        #         optimize = 1,
        #         region_to_fit = (0,147,0,147)
        # )
        # 
        # galaxy_feedme = FeedmeContainer(
        #         path_to_feedme = feedme, 
        #         header = header,
        #         bulge  = bulge,
        #         disk   = disk
        # )
        # 
        # galaxy_feedme.to_file()
        # 
        # #print(galaxy_feedme)
        # process = sp(f"{RUN_GALFIT} {feedme}")
        # 
        # # Open the FITS file
        # fits_file = fits.open(output_filename)
        # 
        # # Extract the image data from the FITS file
        # image_data_fits     = fits_file[0].data - galaxy_feedme.sky.sky_background.value
        # new_image_data_fits = deepcopy(image_data_fits)
        # 
        # #image_data_array = fits_file[1].data
        # 
        # # Close the FITS file
        # fits_file.close()
        
        # # Create a figure and subplots
        # fig, ax1 = plt.subplots(1, 1, figsize=(10, 5))
        # 
        # # Display the FITS image
        # image_fits = ax1.imshow(np.log(1 + image_data_fits), cmap='gray')
        # 
        # # Display the array image
        # #image_array = ax2.imshow(image_data_array, cmap='gray')
        # 
        # # Create a slider widget for the FITS image
        # slider_fits = widgets.FloatSlider(value = 1.0, min = 0.0, max = 2.0, step=0.01, description='FITS Scale:', continuous_update=True)
        # 
        # # Create a slider widget for the array image
        # #slider_array = widgets.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description='Array Scale:', continuous_update=True)
        # 
        # # Function to update the FITS image when slider value changes
        # def update_image_fits(change):
        #     # Update the image data based on the slider value
        #     new_image_data_fits = change.new * np.log(1 + image_data_fits)
        # 
        #     # Update the FITS image with new data
        #     image_fits.set_data(new_image_data_fits)
        # 
        #     # Redraw the figure
        #     fig.canvas.draw()
        # 
        # # Attach the update_image_fits function to the FITS slider's value change event
        # slider_fits.observe(update_image_fits, 'value')
        # 
        # # Attach the update_image_array function to the array slider's value change event
        # #slider_array.observe(update_image_array, 'value')
        # 
        # # Display the slider widgets
        # display(widgets.HBox([slider_fits])) #, slider_array]))
        # 
        # # Add any additional customization you need to the subplots
        # ax1.set_title('FITS Image')
        # ax1.set_xlabel('X-axis')
        # ax1.set_ylabel('Y-axis')
        # 
        # #ax2.set_title('Residual')
        # #ax2.set_xlabel('X-axis')
        # #ax2.set_ylabel('Y-axis')
        # 
        # # Show the plot
        # plt.show()
            
        # x = input()
        count += 1
        
    g_number += 1

Value (690.91) not found in *any* range.
Choosing note: E5


<Figure size 640x480 with 0 Axes>

In [22]:
print(note_dict_ranges)

{'C0': [16.35, 17.32], 'D0': [17.32, 18.35, 19.45], 'E0': [19.45, 20.6], 'F0': [21.83, 23.12], 'G0': [23.12, 24.5, 25.96], 'A0': [25.96, 27.5, 29.14], 'B0': [29.14, 30.87, 32.7], 'C1': [32.7, 34.65], 'D1': [34.65, 36.71, 38.89], 'E1': [38.89, 41.2], 'F1': [43.65, 46.25], 'G1': [46.25, 49.0, 51.91], 'A1': [51.91, 55.0, 58.27], 'B1': [58.27, 61.74, 65.41], 'C2': [65.41, 69.3], 'D2': [69.3, 73.42, 77.78], 'E2': [77.78, 82.41], 'F2': [87.31, 92.5], 'G2': [92.5, 98.0, 103.83], 'A2': [103.83, 110.0, 116.54], 'B2': [116.54, 123.47, 130.81], 'C3': [130.81, 138.59], 'D3': [138.59, 146.83, 155.56], 'E3': [155.56, 164.81], 'F3': [174.61, 185.0], 'G3': [185.0, 196.0, 207.65], 'A3': [207.65, 220.0, 233.08], 'B3': [233.08, 246.94, 261.63], 'C4': [261.63, 277.18], 'D4': [277.18, 293.66, 311.13], 'E4': [311.13, 329.63], 'F4': [349.23, 369.99], 'G4': [369.99, 392.0, 415.3], 'A4': [415.3, 440.0, 466.16], 'B4': [466.16, 493.88, 523.25], 'C5': [523.25, 554.37], 'D5': [554.37, 587.33, 622.25], 'E5': [622.2

In [ ]:
# plt.clf()
# fig,ax = plt.subplots()
# ax.plot(freqs[mask], np.abs(FT[mask])**2)
# plt.show()

In [30]:
#print(list_o_notes)
from collections import Counter
c = Counter(list_o_notes)
print(c)

Counter({'D5': 20, 'G4': 9, 'B4': 7, 'C5': 6, 'E5': 6, 'D4': 4, 'E4': 3, 'G5': 2, 'A4': 2, 'C4': 1})
